In [1]:
import pyreadstat

df, meta = pyreadstat.read_sas7bdat('../data/2022/cy08msp_stu_qqq.sas7bdat', usecols=["CNT", "W_FSTUWT", "PV1MATH", "PV2MATH", "PV3MATH", "PV4MATH", "PV5MATH", "PV6MATH", "PV7MATH", "PV8MATH", "PV9MATH", "PV10MATH"])

In [3]:
import polars as pl
import numpy as np

In [4]:
df["math"] = np.mean([df["PV1MATH"], df["PV2MATH"], df["PV3MATH"], df["PV4MATH"], df["PV5MATH"], df["PV6MATH"], df["PV7MATH"], df["PV8MATH"], df["PV9MATH"], df["PV10MATH"]], axis=0)

In [5]:
df.head()

,CNT,W_FSTUWT,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH,PV6MATH,PV7MATH,PV8MATH,PV9MATH,PV10MATH,math
0,ALB,3.15874,179.583,201.334,222.248,194.943,222.755,213.100,221.021,293.710,226.912,254.745,223.0351
1,ALB,4.34524,308.247,304.444,316.129,234.353,323.448,303.435,333.178,288.888,343.157,329.659,308.4938
2,ALB,7.83860,267.514,343.743,327.157,290.771,316.227,309.451,323.337,307.508,356.244,295.401,313.7353
3,ALB,8.49148,272.649,300.642,226.409,316.835,288.861,321.268,312.063,334.765,283.916,329.915,298.7323
4,ALB,3.70957,435.473,491.174,478.365,454.352,491.031,504.919,468.401,463.913,470.627,499.257,475.7512


In [6]:
total = pl.scan_parquet("../data/combined_all.parquet")

In [7]:
total.filter(pl.col("year") == 2018).head().collect()

country,year,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population,arwu_ranked_num,arwu_sum_score,math,read,science,in_math99,imo_total_score,democracy_eiu
str,i64,f64,f64,f64,f64,f64,u64,u32,f64,f64,f64,f64,f64,f64,f64
"""AFG""",2018,492.090632,-1.688577,null,null,null,36686784,0,0.0,null,null,null,null,0.0,2.97
"""ALB""",2018,5287.660801,4.276312,null,null,null,2866376,0,0.0,437.554623,406.819682,416.779228,0.062738,37.0,5.98
"""DZA""",2018,4171.790388,-0.70795,null,null,null,41927007,0,0.0,null,null,null,null,0.0,3.5
"""ASM""",2018,13195.9359,4.874061,null,null,null,48424,0,0.0,null,null,null,null,0.0,null
"""AND""",2018,42904.811588,-0.00387,null,null,null,75013,0,0.0,null,null,null,null,0.0,null


In [8]:
df.rename({"CNT": "country", "W_FSTUWT":"stu_wgt"}, axis=1)

,country,stu_wgt,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH,PV6MATH,PV7MATH,PV8MATH,PV9MATH,PV10MATH,math
0,ALB,3.15874,179.583,201.334,222.248,194.943,222.755,213.100,221.021,293.710,226.912,254.745,223.0351
1,ALB,4.34524,308.247,304.444,316.129,234.353,323.448,303.435,333.178,288.888,343.157,329.659,308.4938
2,ALB,7.83860,267.514,343.743,327.157,290.771,316.227,309.451,323.337,307.508,356.244,295.401,313.7353
3,ALB,8.49148,272.649,300.642,226.409,316.835,288.861,321.268,312.063,334.765,283.916,329.915,298.7323
4,ALB,3.70957,435.473,491.174,478.365,454.352,491.031,504.919,468.401,463.913,470.627,499.257,475.7512
...,...,...,...,...,...,...,...,...,...,...,...,...,...
613739,UZB,69.40366,300.530,329.387,359.595,314.353,322.403,333.284,311.266,342.431,348.710,308.192,327.0151
613740,UZB,63.18910,265.080,257.582,327.787,311.439,267.167,323.318,273.206,268.522,296.958,273.324,286.4383
613741,UZB,65.47339,420.221,399.911,420.742,432.866,392.747,397.174,421.565,407.357,407.601,418.482,411.8666
613742,UZB,66.15595,343.129,355.853,296.233,335.538,372.287,315.866,335.566,405.146,313.752,307.822,338.1192


In [9]:
df = pl.from_pandas(df)
#.with_columns(pl.lit(2022).alias("year"))

In [10]:
subset = df.rename({"CNT": "country", "W_FSTUWT":"stu_wgt"}).select(["country", "stu_wgt", "math"])
subset.head()

country,stu_wgt,math
str,f64,f64
"""ALB""",3.15874,223.0351
"""ALB""",4.34524,308.4938
"""ALB""",7.8386,313.7353
"""ALB""",8.49148,298.7323
"""ALB""",3.70957,475.7512


In [11]:
bound = np.percentile(subset["math"], 99)
bound

679.2751109999998

In [12]:
by_country = subset.group_by(pl.col("country")).agg(((pl.col("math") * pl.col("stu_wgt")).sum() / pl.col("stu_wgt").sum()).alias("math"), (((pl.col("math") > bound) * 100 * pl.col("stu_wgt")).sum() / pl.col("stu_wgt").sum()).alias("in_math99")).with_columns(pl.lit(2022).alias("year").cast(pl.Int64))

In [13]:
by_country.head()

country,math,in_math99,year
str,f64,f64,i64
"""ISR""",457.896667,1.207422,2022
"""PRY""",337.53975,0.0,2022
"""NOR""",468.44995,0.785227,2022
"""PAN""",356.573504,0.0,2022
"""SRB""",439.876667,0.476665,2022


In [14]:
total = total.join(by_country.lazy(), on=["country", "year"], how="left")

In [15]:
total.collect()

country,year,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population,arwu_ranked_num,arwu_sum_score,math,read,science,in_math99,imo_total_score,democracy_eiu,math_right,in_math99_right
str,i64,f64,f64,f64,f64,f64,u64,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""AFG""",2003,199.643228,0.927029,null,null,null,22645130,0,0.0,null,null,null,null,0.0,null,null,null
"""AFG""",2004,221.830531,-2.497255,null,null,null,23553551,0,0.0,null,null,null,null,0.0,null,null,null
"""AFG""",2005,254.115274,7.321874,null,null,null,24411191,0,0.0,null,null,null,null,0.0,null,null,null
"""AFG""",2006,274.015394,1.084988,null,null,null,25442944,0,0.0,null,null,null,null,0.0,3.06,null,null
"""AFG""",2007,376.318296,11.803383,null,null,null,25903301,0,0.0,null,null,null,null,0.0,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""WLD""",2018,11297.452401,2.144759,null,null,null,7660371127,0,0.0,null,null,null,null,0.0,null,null,null
"""WLD""",2019,11338.150319,1.512061,null,null,null,7741774583,0,0.0,null,null,null,null,0.0,null,null,null
"""WLD""",2020,10904.147614,-4.03007,null,null,null,7820205606,0,0.0,null,null,null,null,0.0,null,null,null


In [16]:
as_pd = total.collect().to_pandas()

In [17]:
as_pd["math"] = np.where(as_pd["year"] == 2022, as_pd["math_right"], as_pd["math"])
as_pd["in_math99"] = np.where(as_pd["year"] == 2022, as_pd["in_math99_right"], as_pd["in_math99"])

In [18]:
as_pd = as_pd.drop(columns=["math_right", "in_math99_right"])

In [19]:
# as_pd.to_parquet("../data/combined-2022-included.parquet")